In [1]:
import io
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data

In [2]:
import pandas as pd
import numpy as np

In [3]:
file_dir = 'E://kaggle//Cdiscount//dataset'

In [4]:
data = bson.decode_file_iter(open(file_dir+'//train_example.bson', 'rb'))

prod_to_category = dict()
for c, d in enumerate(data):
    product_id = d['_id']
    category_id = d['category_id'] # This won't be in Test data'
    
#     print(d['category_id'])
    prod_to_category['id'] = category_id
    
    lst_pic = []
    for e, pic in enumerate(d['imgs']):
        picture = imread(io.BytesIO(pic['picture']))
        
#         picture = np.float32(picture) / 255
#         print((picture.shape))
        lst_pic.append(picture)
        # do something with the picture, etc
    prod_to_category['pictures'] = lst_pic
#     print(len(prod_to_category['pictures']))

In [5]:
(prod_to_category['id'])

1000004085

In [6]:
lst_pic

[array([[[ 0.99607843,  0.99607843,  0.99607843],
         [ 0.99607843,  0.99607843,  0.99607843],
         [ 0.99607843,  0.99607843,  0.99607843],
         ..., 
         [ 0.99607843,  0.99607843,  0.99607843],
         [ 0.99607843,  0.99607843,  0.99607843],
         [ 0.99607843,  0.99607843,  0.99607843]],
 
        [[ 0.99607843,  0.99607843,  0.99607843],
         [ 0.99607843,  0.99607843,  0.99607843],
         [ 0.99607843,  0.99607843,  0.99607843],
         ..., 
         [ 0.99607843,  0.99607843,  0.99607843],
         [ 0.99607843,  0.99607843,  0.99607843],
         [ 0.99607843,  0.99607843,  0.99607843]],
 
        [[ 0.99607843,  0.99607843,  0.99607843],
         [ 0.99607843,  0.99607843,  0.99607843],
         [ 0.99607843,  0.99607843,  0.99607843],
         ..., 
         [ 0.99607843,  0.99607843,  0.99607843],
         [ 0.99607843,  0.99607843,  0.99607843],
         [ 0.99607843,  0.99607843,  0.99607843]],
 
        ..., 
        [[ 0.99607843,  0.996078

# Building Model

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [6]:
num_classes=10
epochs = 10

In [7]:
model = Sequential()
# Convolutional Layer
model.add(Conv2D(32, (3,3), input_shape = (32,32,3), activation='relu'))

model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax')) 


lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])